In [1]:
# baseline改进，使用特征osv          ?osv变成float
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
train = pd.read_csv('./train.csv')
test1 = pd.read_csv('./test1.csv')

In [4]:
features = train.drop(['Unnamed: 0','label'],axis = 1)
labels = train['label']

Index(['android_id', 'apptype', 'carrier', 'dev_height', 'dev_ppi',
       'dev_width', 'lan', 'media_id', 'ntt', 'os', 'osv', 'package', 'sid',
       'timestamp', 'version', 'fea_hash', 'location', 'fea1_hash',
       'cus_type'],
      dtype='object')

In [5]:
#找object
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500000 entries, 0 to 499999
Data columns (total 21 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Unnamed: 0  500000 non-null  int64  
 1   android_id  500000 non-null  int64  
 2   apptype     500000 non-null  int64  
 3   carrier     500000 non-null  float64
 4   dev_height  500000 non-null  float64
 5   dev_ppi     500000 non-null  float64
 6   dev_width   500000 non-null  float64
 7   label       500000 non-null  int64  
 8   lan         316720 non-null  object 
 9   media_id    500000 non-null  int64  
 10  ntt         500000 non-null  float64
 11  os          500000 non-null  object 
 12  osv         493439 non-null  object 
 13  package     500000 non-null  int64  
 14  sid         500000 non-null  int64  
 15  timestamp   500000 non-null  float64
 16  version     500000 non-null  object 
 17  fea_hash    500000 non-null  object 
 18  location    500000 non-null  int64  
 19  fe

In [6]:
#找缺省值
train.isnull().sum()

Unnamed: 0         0
android_id         0
apptype            0
carrier            0
dev_height         0
dev_ppi            0
dev_width          0
label              0
lan           183280
media_id           0
ntt                0
os                 0
osv             6561
package            0
sid                0
timestamp          0
version            0
fea_hash           0
location           0
fea1_hash          0
cus_type           0
dtype: int64

In [7]:
#查看字段唯一值个数
for feature in features.columns:
    print(feature, train[feature].nunique())

android_id 362258
apptype 89
carrier 5
dev_height 798
dev_ppi 92
dev_width 346
lan 21
media_id 284
ntt 8
os 2
osv 154
package 1950
sid 500000
timestamp 500000
version 22
fea_hash 402980
location 332
fea1_hash 4959
cus_type 58


In [8]:
#carrier属于类别特征，不是数值特征
features['carrier'].value_counts()

 46000.0    359409
 46001.0     43390
 0.0         40652
 46003.0     32294
-1.0         24255
Name: carrier, dtype: int64

In [9]:
train['osv'].value_counts()

8.1.0            151419
9                 71332
6.0.1             31714
7.1.1             26503
9.0.0             24385
                  ...  
8.0.2                 1
Android_5.0.1         1
9.1                   1
Android_4.4.3         1
Android 7.1           1
Name: osv, Length: 154, dtype: int64

In [11]:
#类别特征
cate_features = ['apptype','carrier','ntt','version','location','cus_type']
#不需要入模'os'（实际是唯一的，都是安卓）,'lan','sid'
remove_list = ['os','lan','sid']
col = features.columns.tolist()
for i in remove_list:
    col.remove(i)
col

['android_id',
 'apptype',
 'carrier',
 'dev_height',
 'dev_ppi',
 'dev_width',
 'media_id',
 'ntt',
 'osv',
 'package',
 'timestamp',
 'version',
 'fea_hash',
 'location',
 'fea1_hash',
 'cus_type']

In [12]:
#特征筛选
features = features[col]
features

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,media_id,ntt,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type
0,316361,1199,46000.0,0.0,0.0,0.0,104,6.0,9,18,1.559893e+12,8,2135019403,0,2329670524,601
1,135939,893,0.0,0.0,0.0,0.0,19,6.0,8.1,0,1.559994e+12,4,2782306428,1,2864801071,1000
2,399254,821,0.0,760.0,0.0,360.0,559,0.0,8.1.0,0,1.559837e+12,0,1392806005,2,628911675,696
3,68983,1004,46000.0,2214.0,0.0,1080.0,129,2.0,8.1.0,0,1.560042e+12,0,3562553457,3,1283809327,753
4,288999,1076,46000.0,2280.0,0.0,1080.0,64,2.0,8.0.0,0,1.559867e+12,5,2364522023,4,1510695983,582
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
499995,392477,1028,46000.0,1920.0,3.0,1080.0,144,6.0,7.1.2,25,1.559834e+12,7,861755946,79,140647032,373
499996,346134,1001,0.0,1424.0,0.0,720.0,29,2.0,8.1.0,0,1.559814e+12,3,1714444511,23,2745131047,525
499997,499635,761,46000.0,1280.0,0.0,720.0,54,6.0,6.0.1,9,1.559676e+12,0,3843262581,25,1326115882,810
499998,239786,917,46001.0,960.0,0.0,540.0,109,2.0,5.1.1,0,1.559840e+12,0,1984296118,225,1446741112,772


In [19]:
#对osv进行特征编码LabelEncoder
#还可以转换为Float,需要数据清洗
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
#需要将训练集和测试集合并，然后统一做LabelEncoder
all_df = pd.concat([train,test1])
all_df['osv'] = all_df['osv'].astype('str')
all_df['osv'] = le.fit_transform(all_df['osv'])

In [20]:
# 特征变换
features['fea_hash'].value_counts()

68083895      110
235856055      99
51306679       91
2815114810     77
16777343       60
             ... 
2856033499      1
2175192677      1
2989345463      1
3239274719      1
3373383863      1
Name: fea_hash, Length: 402980, dtype: int64

In [21]:
features['fea_hash'].map(lambda x: len(str(x))).value_counts()
features['fea1_hash'].map(lambda x: len(str(x))).value_counts()

10    391669
9      99347
8       8977
7          6
5          1
Name: fea1_hash, dtype: int64

In [22]:
# 对于数值过大的异常值，设置为0
features['fea_hash'] = features['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
features['fea1_hash'] = features['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

In [23]:
#对数据清洗，将V3=>3,V1=>1,V6=>6,V2=>2
#针对version，非数值类型 设置0
features['version'] = features['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
features['osv'] = all_df[all_df['label'].notnull()]['osv']

In [24]:
import lightgbm as lgb
# 使用LightGBM训练
# 相比XGBoost的改进：对类别特征的处理，不需要one-hot
clf = lgb.LGBMClassifier()
clf.fit(features,labels,categorical_feature = cate_features)

LGBMClassifier()

In [25]:
#测试集做预测,保持与features中的columns一致
test_fea = test1[features.columns]

test_fea['fea_hash'] = test_fea['fea_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))
test_fea['fea1_hash'] = test_fea['fea1_hash'].map(lambda x: 0 if len(str(x))>16 else int(x))

test_fea['version'] = test_fea['version'].map(lambda x: int(x) if str(x).isdigit() else 0)
test_fea['osv'] = all_df[all_df['label'].isnull()]['osv']

In [26]:
a = pd.DataFrame(test1['sid'])
a['label'] =  clf.predict(test_fea)
a.to_csv('version2.csv',index = False)

In [64]:
a

,sid,label
0,1440682,0
1,1606824,1
2,1774642,0
3,1742535,0
4,1689686,1
...,...,...
149995,1165373,1
149996,1444115,1
149997,1134378,1
149998,1700238,1
